In [1]:
from xgboost import XGBClassifier, cv, DMatrix
import optuna
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score, precision_score, recall_score
from google.cloud import datastore, bigquery, secretmanager
from sklearn.model_selection import *
import numpy as np
import pandas as pd
import os, warnings
import tensorflow as tf
import joblib, json
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, RepeatVector, Input, Reshape, Flatten
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from tensorflow.keras.models import Model
from data_preprocessing import *
nltk.download('punkt')  # For tokenization
nltk.download('stopwords')  # Download stop words
nltk.download('wordnet')  # For lemmatization

/home/hemanth/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-10 20:01:56.173132: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 20:01:56.173170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 20:01:56.174190: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-10 20:01:56.180648: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is opt

True

In [2]:
df = pd.read_excel(open('Consolidated.xlsx', 'rb'), sheet_name='Consolidated')
df_final_test = df.query('Foldername == "fold1"')
df = df.query('Foldername != "fold1"')
df['Review'] = df['Review'].apply(preprocess_text)

In [3]:
vectorizer = TfidfVectorizer(max_features=1024)
le = LabelEncoder()
le.fit(df['Target'])
vectorizer.fit(df['Review'])

In [6]:
tfidf_vectors = vectorizer.transform(df['Review']).toarray()

In [7]:
@tf.keras.saving.register_keras_serializable()
class Autoencoder(Model):
  def __init__(self, latent_dim, shape):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.shape = shape
    self.encoder = tf.keras.Sequential([
      Dense(1024, input_dim=shape[1], activation='relu'),
      Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      Dense(128, input_dim=latent_dim, activation='relu'),
      Dense(shape[1], activation='relu'),
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [8]:
shape = tfidf_vectors.shape
latent_dim = 32
autoencoder = Autoencoder(latent_dim, shape)
autoencoder.compile(optimizer='adam', loss='mse')

2024-04-10 16:21:04.882563: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9920 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:65:00.0, compute capability: 8.6


In [9]:
autoencoder.fit(tfidf_vectors, tfidf_vectors,
                epochs=20,
                batch_size=64,
                shuffle=True)

Epoch 1/20


2024-04-10 16:21:06.916190: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-10 16:21:07.224454: I external/local_xla/xla/service/service.cc:168] XLA service 0x7332a663e240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-10 16:21:07.224480: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-04-10 16:21:07.229141: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-10 16:21:07.243848: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1712746267.334700   57398 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


45/45 [==============================] - 2s 2ms/step - loss: 9.2342e-04
Epoch 2/20
45/45 [==============================] - 0s 2ms/step - loss: 9.0358e-04
Epoch 3/20
45/45 [==============================] - 0s 2ms/step - loss: 8.9694e-04
Epoch 4/20
45/45 [==============================] - 0s 2ms/step - loss: 8.8405e-04
Epoch 5/20
45/45 [==============================] - 0s 2ms/step - loss: 8.6831e-04
Epoch 6/20
45/45 [==============================] - 0s 2ms/step - loss: 8.4799e-04
Epoch 7/20
45/45 [==============================] - 0s 2ms/step - loss: 8.2374e-04
Epoch 8/20
45/45 [==============================] - 0s 2ms/step - loss: 7.9580e-04
Epoch 9/20
45/45 [==============================] - 0s 2ms/step - loss: 7.6691e-04
Epoch 10/20
45/45 [==============================] - 0s 2ms/step - loss: 7.3844e-04
Epoch 11/20
45/45 [==============================] - 0s 2ms/step - loss: 7.1067e-04
Epoch 12/20
45/45 [==============================] - 0s 2ms/step - loss: 6.8429e-04
Epoch 13/20


In [10]:
df_encoded = pd.DataFrame(autoencoder.encoder(tfidf_vectors).numpy())

In [57]:
X, X_tt, y, y_tt = train_test_split(df_encoded, le.transform(df['Target']), train_size=0.9,  random_state=42)

In [58]:
def optuna_objective_xgboost(trial):
    # Create model
    alpha = trial.suggest_float('alpha', 0.1, 2)
    gamma=trial.suggest_float('gamma', 0.1, 0.9)
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    max_depth = trial.suggest_int('max_depth', 3, 8)
    XGB_clf = XGBClassifier(objective='multi:softmax', gamma=gamma, alpha=alpha, max_depth=max_depth, enable_categorical=True)
    skf = KFold(n_splits=8)
    cv_scores = cross_val_score(XGB_clf, X, y, scoring='f1_macro', cv=skf, n_jobs=8)
    return cv_scores.mean()

In [59]:
study = optuna.create_study(direction='maximize')
study.optimize(optuna_objective_xgboost, n_trials=50, n_jobs=6)

[I 2024-04-10 18:08:06,187] A new study created in memory with name: no-name-2e0142ba-9f48-4f3d-88fa-0b97fed75eac
[I 2024-04-10 18:08:41,528] Trial 2 finished with value: 0.8930176844445777 and parameters: {'alpha': 1.610949484240721, 'gamma': 0.454175653412464, 'n_estimators': 446, 'max_depth': 5}. Best is trial 2 with value: 0.8930176844445777.
[I 2024-04-10 18:08:41,926] Trial 0 finished with value: 0.8950447303753863 and parameters: {'alpha': 1.673241857641758, 'gamma': 0.4220289551569193, 'n_estimators': 276, 'max_depth': 3}. Best is trial 0 with value: 0.8950447303753863.
[I 2024-04-10 18:08:42,271] Trial 4 finished with value: 0.9107481019253687 and parameters: {'alpha': 0.5555032450047074, 'gamma': 0.14989193976578347, 'n_estimators': 439, 'max_depth': 3}. Best is trial 4 with value: 0.9107481019253687.
[I 2024-04-10 18:08:42,385] Trial 3 finished with value: 0.9135873591729808 and parameters: {'alpha': 0.3860923579161115, 'gamma': 0.24406823861692148, 'n_estimators': 408, 'max

In [60]:
XGB_clf = XGBClassifier(objective='multi:softmax', enable_categorical=True, **study.best_params)

In [61]:
XGB_clf.fit(X, y)

XGBClassifier(alpha=0.12636021805280295, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=0.10619969093244841, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=383, n_jobs=None,
              num_parallel_tree=None, ...)

In [62]:
y_pred = XGB_clf.predict(X_tt)
print('Accuracy Score predict: ', f1_score(y_tt, y_pred, average='macro'))

Accuracy Score predict:  0.9588447518226306


In [63]:
print(classification_report(y_tt, y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99        67
           1       0.97      0.97      0.97        76
           2       0.94      0.92      0.93        78
           3       0.95      0.94      0.95        67

    accuracy                           0.96       288
   macro avg       0.96      0.96      0.96       288
weighted avg       0.96      0.96      0.96       288



In [18]:
save_models(le, vectorizer, autoencoder, XGB_clf)

## Test model with unseen data and custom reviews

In [3]:
l_le, l_vectorizer, l_autoencoder, l_XGB_clf = load_models()

2024-04-10 20:02:15.722167: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1038 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:65:00.0, compute capability: 8.6


In [4]:
hh = l_vectorizer.transform(df_final_test['Review'].apply(preprocess_text)).toarray()
t_df_encoded = pd.DataFrame(l_autoencoder.encoder(hh).numpy())
t_pred = l_XGB_clf.predict(t_df_encoded)

2024-04-10 20:02:18.764874: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [5]:
f1_score(l_le.transform(df_final_test['Target']), t_pred, average='macro')

0.96875

In [7]:
def predict_for_custom_review(text):
    ff = preprocess_text(text)
    hh = l_vectorizer.transform([ff]).toarray()
    df_encoded = pd.DataFrame(l_autoencoder.encoder(hh).numpy())
    y_pred = l_XGB_clf.predict(df_encoded)
    return l_le.inverse_transform(y_pred)[0]

In [25]:
text = """I will say the hotel was nice and the staff was friendly. The reason for my review is hidden fees. I thought I had paid for my hotel fully through booking online. Two days later I saw an additional charge on my credit card. It was for a “destination fee”. I was not informed of this ahead of time and find this to be dishonest."""

In [26]:
predict_for_custom_review(text)

'True Negative'

In [9]:
import gradio as gr
interface = gr.Interface(fn=predict_for_custom_review, 
                         inputs='text', 
                         outputs="label", 
                         title="Fake Review Detector")
if __name__ == "__main__":
    interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
